In [1]:
from openai import OpenAI
from FlagEmbedding import BGEM3FlagModel
import pandas as pd
import transformers 
import os

# Don't forget your key. :)

In [18]:
df_en = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\train_ds\mushroom.en-train_nolabel.v1.jsonl', lines=True) #Cambiar para directorio local
df_es = pd.read_json(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\train_ds\mushroom.es-train_nolabel.v1.jsonl', lines=True)
df_en.head(5)

,lang,model_id,model_input,model_output_text,model_output_logits,model_output_tokens
0,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
1,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids have at least ...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
2,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
3,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."
4,EN,togethercomputer/Pythia-Chat-Base-7B,Do all arthropods have antennae?,"Yes, all insects and arachnids (including spi...","[-2.57427001, 5.1865358353, 5.4173498154, 2.32...","[ĠYes, ,, Ġall, Ġinsects, Ġand, Ġar, ach, n, i..."


In [19]:
embs_aux = pd.read_csv(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\eng_embs_singulares.csv')
embs_aux.head(5)

,Unnamed: 0,Embedding,Texto,Keywords Pregunta
0,0,[-0.04245 -0.004425 0.02083 ... -0.011086 ...,"The Arts and Humanities Citation Index (AHCI),...",Is the Arts and Humanities Citation Index stil...
1,1,[ 0.01083 -0.005505 0.00785 ... 0.02777 ...,Valrico is a census-designated place (CDP) in ...,"What was Valrico, Florida known as before the ..."
2,2,[ 0.01604 -0.01588 -0.05432 ... -0.01395 0.03...,"Bleeding Kansas, Bloody Kansas, or the Border ...",When did the Bleeding Kansas civil confrontati...
3,3,[-0.03955 -0.04813 -0.01185 ... -0.014534 -...,"Tomy Company, Ltd. (株式会社タカラトミー, Kabushikigaish...",When did the merger of Takara and Tomy take pl...
4,4,[-0.01819 0.04468 -0.013504 ... 0.03748 -...,Count Carl Axel Löwenhielm (3 November 1772 – ...,Who was the father of Carl Löwenhielm?


In [2]:
# FUNCIONES

def get_questions(path):
    """
    Returns dataset questions in a list.

    path = str; File location.
    """
    dataset = pd.read_json(path, lines = True)
    questions  = dataset["model_input"]
    return [_ for _ in questions]

def get_embs(path):
    """
    Returns embeddings as a pandas DataFrame.

    path = str; File location.
    """
    embs = pd.read_csv(path)
    embs = embs.drop(columns = ["Unnamed: 0"])
    embs_text = embs["Texto"]
    embs_vec = embs["Embedding"]
    embs_txt = [_ for _ in embs_text]
    return embs_txt, embs_vec

def gen_answer(ques, retr):
    """
    Generates GPT based answer given a question and a context window.

    ques = str; Question extracted from the task dataset.
    retr = str; Context extracted from embeddings.
    """
    client = OpenAI()
    prompt = f"""
        You are a bot that answers trivia questions.
        Be brief, answer in short sentences highlighting important information.

        This is the trivia question you need to answer:
        {ques}.

        This is text that you should use to answer the question:
        {retr}.
    """

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {
                "role": "user",
                "content": prompt,
            },
        ],
        temperature = 0.1,
    ).choices[0].message
    
    return response.content

In [21]:
en_questions = get_questions(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\train_ds\mushroom.en-train_nolabel.v1.jsonl')
es_questions = get_questions(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\train_ds\mushroom.es-train_nolabel.v1.jsonl')
print(en_questions[0], es_questions[0])

Do all arthropods have antennae? ¿Dónde está ubicado Cilleruelo de San Mamés?


In [22]:
embs_en_single = get_embs(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\eng_embs_singulares.csv')[0]
embs_es_single = get_embs(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\es_embs_singulares.csv')[0]
for _ in embs_en_single[0:2]:
    print(_)
    print("-------")

The Arts and Humanities Citation Index (AHCI), also known as Arts and Humanities Search, is a citation index, with abstracting and indexing for more than 1,700 arts and humanities academic journals, and coverage of disciplines that includes social and natural science journals. Part of this database is derived from Current Contents.
Subjects covered are the arts, humanities, language (including linguistics), poetry, music, classical works, history, oriental studies, philosophy, archaeology, architecture, religion, television, theater, and radio.
Coverage includes articles, letters, editorials, meeting abstracts, errata, poems, short stories, plays, music scores, excerpts from books, chronologies, bibliographies and filmographies, as well as citations to reviews of books, films, music, and theatrical performances.
This database can be accessed online through Web of Science. It provides access to current and retrospective bibliographic information and cited references. It also covers indi

In [26]:
aux_list = embs_aux["Keywords Pregunta"]
for i in range(3):
    ans_aux = gen_answer(aux_list[i], embs_aux["Texto"][i])
    print(aux_list[i], ans_aux)

Is the Arts and Humanities Citation Index still maintained? Yes, the Arts and Humanities Citation Index (AHCI) is still maintained. It provides access to over 1,700 arts and humanities journals and is updated regularly. The database can be accessed online through Web of Science.
What was Valrico, Florida known as before the Civil War? Before the Civil War, Valrico, Florida was known as "Baker's Station."
When did the Bleeding Kansas civil confrontations take place? The Bleeding Kansas civil confrontations took place between 1854 and 1859. This period was marked by violent conflicts over the legality of slavery in the Kansas Territory.


In [11]:
print(f"EN: Conjunto de preguntas base: {len(en_questions)}, Conjunto de preguntas SIN repeticiones: {len(set(en_questions))}, Conjunto de Embeddings: {len(embs_en_single)}")
print(f"ES: Conjunto de preguntas base: {len(es_questions)}, Conjunto de preguntas SIN repeticiones: {len(set(es_questions))}, Conjunto de Embeddings: {len(embs_es_single)}")

EN: Conjunto de preguntas base: 809, Conjunto de preguntas SIN repeticiones: 49, Conjunto de Embeddings: 49
ES: Conjunto de preguntas base: 492, Conjunto de preguntas SIN repeticiones: 50, Conjunto de Embeddings: 50


In [3]:
def gpt_full(emb_dataset):
    preguntas = emb_dataset["Keywords Pregunta"]
    contexto = emb_dataset["Texto"]
    gpt_answer = []
    for i in range(len(contexto)):
        answer = gen_answer(preguntas[i], contexto[i])
        gpt_answer.append(answer)
    dic = {'Pregunta': preguntas, 'Respuesta GPT': gpt_answer, 'Contexto': contexto}
    full_ds = pd.DataFrame(data=dic)
    return full_ds

In [5]:
val_embs = pd.read_csv(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\embeddings\val_embs\en_embs.csv')
val_embs = val_embs.drop(columns = ["Unnamed: 0"])
val_embs.head(5)

,Embedding,Texto,Keywords Pregunta
0,[-0.01855 0.02477 -0.04358 ... -0.01863 ...,The 26th Biathlon World Championships were hel...,In which city were the 26th biathlon world cha...
1,[-0.02757 -0.0263 -0.04037 ... -0.0327 -0.03...,"David John Franco (born June 12, 1985) is an A...",Who did Dave Franco play in Scrubs?
2,[-0.03198 0.0355 -0.01357 ... 0.02058 -0.03...,The province of Alessandria (Italian: provinci...,Which municipalities does the Italian commune ...
3,[-0.0385 0.03262 -0.02072 ... 0.004704 ...,"SDG 4, or Sustainable Development Goal 4, is a...",What is the UN Sustainable Development Goal 11...
4,[-0.0424 -0.004433 0.02086 ... -0.011086 ...,"The Arts and Humanities Citation Index (AHCI),...",Is the Arts and Humanities Citation Index stil...


In [6]:
%%time
dataset_val = gpt_full(val_embs)
dataset_val

CPU times: total: 14 s
Wall time: 15min 59s


,Pregunta,Respuesta GPT,Contexto
0,In which city were the 26th biathlon world cha...,The 26th Biathlon World Championships were hel...,The 26th Biathlon World Championships were hel...
1,Who did Dave Franco play in Scrubs?,Dave Franco played the character Cole in the f...,"David John Franco (born June 12, 1985) is an A..."
2,Which municipalities does the Italian commune ...,Ponzone borders the following municipalities: ...,The province of Alessandria (Italian: provinci...
3,What is the UN Sustainable Development Goal 11...,The provided text does not contain information...,"SDG 4, or Sustainable Development Goal 4, is a..."
4,Is the Arts and Humanities Citation Index stil...,"Yes, the Arts and Humanities Citation Index (A...","The Arts and Humanities Citation Index (AHCI),..."
5,What is the home stadium of FC Zhenis?,The home stadium of FC Zhenis is the K.Munaitp...,FC Zhenis (Kazakh: Жеңіс футбол клубы) is a Ka...
6,"If today is 14th October, and it is not a leap...",If today is October 14 and it is not a leap ye...,"February 29 is a leap day (or ""leap year day"")..."
7,When was the municipality of Delley-Portalban ...,The municipality of Delley-Portalban was creat...,Delley-Portalban (French pronunciation: [dɛlɛ ...
8,Who was the first physician to treat patients ...,The first physician to treat patients for cere...,Bell's palsy is a type of facial paralysis tha...
9,What’s the real name of Edu Manga?,The real name of Edu Manga is Eduardo Antônio ...,Cerebral palsy (CP) is a group of movement dis...


In [8]:
dataset_val.to_csv(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\GPT_Answers\QAC_Val.csv')

In [28]:
%%time
dataset_sexo = gpt_full(embs_aux)
dataset_sexo

CPU times: total: 19.1 s
Wall time: 15min 24s


,Pregunta,Respuesta GPT,Contexto
0,Is the Arts and Humanities Citation Index stil...,"Yes, the Arts and Humanities Citation Index (A...","The Arts and Humanities Citation Index (AHCI),..."
1,"What was Valrico, Florida known as before the ...","Before the Civil War, Valrico, Florida was kno...",Valrico is a census-designated place (CDP) in ...
2,When did the Bleeding Kansas civil confrontati...,The Bleeding Kansas civil confrontations took ...,"Bleeding Kansas, Bloody Kansas, or the Border ..."
3,When did the merger of Takara and Tomy take pl...,The merger of Takara and Tomy took place in 2006.,"Tomy Company, Ltd. (株式会社タカラトミー, Kabushikigaish..."
4,Who was the father of Carl Löwenhielm?,The father of Carl Löwenhielm was King Charles...,Count Carl Axel Löwenhielm (3 November 1772 – ...
5,Which town is Hüffelsheim partnered with?,The text does not specify a partner town for H...,Hüffelsheim is an Ortsgemeinde – a municipalit...
6,Which municipalities does the Italian commune ...,Ponzone borders the following municipalities: ...,The province of Alessandria (Italian: provinci...
7,When was the Swedish Navy founded?,The Swedish Navy was founded in 1522 under Kin...,The Swedish Navy (Swedish: Svenska marinen) is...
8,What’s the real name of Edu Manga?,The real name of Edu Manga is Eduardo Antônio ...,This is a list of active ships of the Royal Sw...
9,In which city were the 26th biathlon world cha...,The 26th Biathlon World Championships were hel...,The 26th Biathlon World Championships were hel...


In [30]:
dataset_sexo.to_csv(r'C:\Users\FLopezP\Documents\GitHub\Mu-SHROOM-GIL\Datasets\GPT_Answers\QuestionAnswerContext.csv')

In [19]:
# ¿Cómo vergas vamos a comparar y evaluar estas chingaderas?

**Ideas de comparación**

https://huggingface.co/fractalego/fact-checking

https://huggingface.co/Dzeniks/roberta-fact-check

https://github.com/Cartus/Automated-Fact-Checking-Resources

https://aclanthology.org/2022.tacl-1.11/